# EXP 1-Random

In [1]:
import numpy as np
import random
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from nupic.bindings.algorithms import TemporalMemory as TM
from htmresearch.support.neural_correlations_utils import *
    
uintType = "uint32"
random.seed(1)

In [2]:
symbolsPerSequence = 10
numSequences = 1000
epochs = 5
totalTS = epochs * numSequences * symbolsPerSequence

In [3]:
tm = TM(columnDimensions = (2048,),
    cellsPerColumn=8,
    initialPermanence=0.21,
    connectedPermanence=0.3,
    minThreshold=15,
    maxNewSynapseCount=40,
    permanenceIncrement=0.1,
    permanenceDecrement=0.1,
    activationThreshold=15,
    predictedSegmentDecrement=0.01,
    )

sparsity = 0.02
sparseCols = int(tm.numberOfColumns() * sparsity)

## Feed sequences to the TM

In [4]:
allSequences = []
spikeTrains = np.zeros((tm.numberOfCells(), totalTS), dtype = "uint32")
columnUsage = np.zeros(tm.numberOfColumns(), dtype="uint32")
ts = 0

entropyX = []
entropyY = []

negPCCX_cells = []
negPCCY_cells = []

negPCCX_cols = []
negPCCY_cols = []

# Randomly generate the indices of the columns to keep track during simulation time
colIndicesSmall = np.random.permutation(tm.numberOfColumns())[0:4] # keep track of 4 columns
colIndicesLarge = np.random.permutation(tm.numberOfColumns())[0:125] # keep track of 4 columns

for s in range(numSequences):
    sequence = generateRandomSequence(symbolsPerSequence, tm.numberOfColumns(), sparsity)
    if s > 0 and s % 50 == 0:
        print str(s) + " sequences processed"
        
    for t in range(epochs):
        for symbol in range(symbolsPerSequence):
            tm.compute(sequence[symbol], learn=True)
            for cell in tm.getActiveCells():
                spikeTrains[cell, ts] = 1            
            
            # Obtain active columns:
            activeColumnsIndices = [tm.columnForCell(i) for i in tm.getActiveCells()]
            currentColumns = [1 if i in activeColumnsIndices else 0 for i in range(tm.numberOfColumns())]
            for col in np.nonzero(currentColumns)[0]:
                columnUsage[col] += 1            
            
            if ts > 0 and ts % int(totalTS * 0.1) == 0:
                print "++ Analyzing correlations (cells at random) ++"                
                subSpikeTrains = subSample(spikeTrains, 1000, tm.numberOfCells(), ts)
                (corrMatrix, numNegPCC) = computePWCorrelations(subSpikeTrains, removeAutoCorr=True)
                negPCCX_cells.append(s)
                negPCCY_cells.append(numNegPCC)                
                print "++ Generating histogram ++"
                bins = 100
                plt.hist(corrMatrix.ravel(), bins, alpha=0.5)                
                # Set range for plot appropriately!
                plt.xlim(-0.05,0.1)
                plt.xlabel("PCC")
                plt.ylabel("Frequency")
                plt.savefig("cellsHist_" + str(ts))
                plt.close()
                print "++ Computing entropy ++"
                entropyX.append(ts)
                entropyY.append(computeEntropy(subSpikeTrains))                
                
                print "++ Analyzing correlations (whole columns) ++"
                ### First the LARGE subsample of columns:
                subSpikeTrains = subSampleWholeColumn(spikeTrains, colIndicesLarge, tm.getCellsPerColumn(), ts)
                (corrMatrix, numNegPCC) = computePWCorrelations(subSpikeTrains, removeAutoCorr=True)
                negPCCX_cols.append(s)
                negPCCY_cols.append(numNegPCC)                
                print "++ Generating histogram ++"
                bins = 50
                plt.hist(corrMatrix.ravel(), bins, alpha=0.5)
                plt.xlabel("PCC")
                plt.ylabel("Frequency")
                plt.savefig("colsHist_" + str(ts))
                plt.close()                 
                ### For the heatmap we consider the SMALL subsample in order to understand what's going on
                subSpikeTrains = subSampleWholeColumn(spikeTrains, colIndicesSmall, tm.getCellsPerColumn(), ts)
                (corrMatrix, numNegPCC) = computePWCorrelations(subSpikeTrains, removeAutoCorr=True)                
                print "++ Generating heatmap ++"
                plt.imshow(corrMatrix, cmap='spectral', interpolation='nearest')
                cb = plt.colorbar()
                cb.set_label('PCC')
                plt.savefig("colsHeatMap_" + str(ts))
                plt.close()
            ts += 1
    allSequences.append(sequence)
print "***All sequences processed!***"

50 sequences processed
100 sequences processed
++ Analyzing correlations (cells at random) ++
++ Generating histogram ++
++ Computing entropy ++
++ Analyzing correlations (whole columns) ++
++ Generating histogram ++
++ Generating heatmap ++
150 sequences processed
200 sequences processed
++ Analyzing correlations (cells at random) ++
++ Generating histogram ++
++ Computing entropy ++
++ Analyzing correlations (whole columns) ++
++ Generating histogram ++
++ Generating heatmap ++
250 sequences processed
300 sequences processed
++ Analyzing correlations (cells at random) ++
++ Generating histogram ++
++ Computing entropy ++
++ Analyzing correlations (whole columns) ++
++ Generating histogram ++
++ Generating heatmap ++
350 sequences processed
400 sequences processed
++ Analyzing correlations (cells at random) ++
++ Generating histogram ++
++ Computing entropy ++
++ Analyzing correlations (whole columns) ++
++ Generating histogram ++
++ Generating heatmap ++
450 sequences processed
500 s

In [5]:
# plot trace of negative PCCs
plt.plot(negPCCX_cells, negPCCY_cells)
plt.xlabel("Time")
plt.ylabel("Negative PCC Count")
plt.savefig("negPCCTrace_cells")
plt.close()

plt.plot(negPCCX_cols, negPCCY_cols)
plt.xlabel("Time")
plt.ylabel("Negative PCC Count")
plt.savefig("negPCCTrace_cols")
plt.close()

In [6]:
# plot entropy
plt.plot(entropyX, entropyY)
plt.xlabel("Time")
plt.ylabel("Entropy")
plt.savefig("entropyTM")
plt.close()

In [7]:
plt.hist(columnUsage)
plt.xlabel("Number of times active")
plt.ylabel("Number of columns")
plt.savefig("columnUsage")
plt.close()

## ISI analysis (with Poisson model too)

In [ ]:
subSpikeTrains = subSample(spikeTrains, 1000, tm.numberOfCells(), totalTS)

In [ ]:
isi = computeISI(subSpikeTrains)

In [ ]:
# Print ISI distribution of TM

#bins = np.linspace(np.min(isi), np.max(isi), 50)
bins = 100
plt.hist(isi, bins)
# plt.xlim(0,4000)
# plt.xlim(89500,92000)
plt.xlabel("ISI")
plt.ylabel("Frequency")
plt.savefig("isiTM")
plt.close()

In [ ]:
firingRate = 8
pSpikeTrain = poissonSpikeGenerator(firingRate,np.shape(subSpikeTrains)[1],np.shape(subSpikeTrains)[0])

In [ ]:
isi = computeISI(pSpikeTrain)

In [ ]:
# Print ISI distribution of Poisson model

#bins = np.linspace(np.min(isi), np.max(isi), 50)
bins = 100
plt.hist(isi, bins)
# plt.xlim(0,4000)
# plt.xlim(89500,92000)
plt.xlabel("ISI")
plt.ylabel("Frequency")
plt.savefig("isiPOI")
plt.close()

## Raster Plots

In [ ]:
subSpikeTrains = subSample(spikeTrains, 100, tm.numberOfCells(), -1)

In [ ]:
rasterPlot(subSpikeTrains, "TM")

In [ ]:
pSpikeTrain = poissonSpikeGenerator(firingRate,np.shape(subSpikeTrains)[1],np.shape(subSpikeTrains)[0])

In [ ]:
rasterPlot(pSpikeTrain, "Poisson")

## Quick Accuracy Test

In [ ]:
simpleAccuracyTest("random", tm, allSequences)

## Elad Plot

In [ ]:
# Sample from both TM_SpikeTrains and Poisson_SpikeTrains. 10 cells for 1000 (?) timesteps
wordLength = 10
firingRate = 8 # This parameter is for the Poisson_SpikeTrains and should be taken experimentally

subSpikeTrains = subSample(spikeTrains, wordLength, tm.numberOfCells(), totalTS)
pSpikeTrain = poissonSpikeGenerator(firingRate,np.shape(subSpikeTrains)[1],np.shape(subSpikeTrains)[0])

In [ ]:
#generate all 2^10 binary strings
binaryStrings = list(itertools.product([0, 1], repeat=wordLength))

In [ ]:
# Now let's do the same experiment but with all 2^10 binary strings
x = [] #observed
y = [] #predicted by random model
for i in range(2**wordLength):
    if i == 0:
        continue
    if i % 100 == 0:
        print str(i) + " words processed"
    binaryWord = np.array(binaryStrings[i], dtype="uint32")
    x.append(countInSample(binaryWord, subSpikeTrains))
    y.append(countInSample(binaryWord, pSpikeTrain))
print "**All words processed**"

In [ ]:
plt.loglog(x, y, 'bo',basex=10)
plt.xlabel("Observed")
plt.ylabel("Predicted")
plt.plot(x,x,'k-')
plt.xlim(0,np.max(x))
# plt.xlim(0,20)
# plt.ylim(0,20)
# plt.show()
plt.savefig("eladPlot")
plt.close()

## Save TM

In [ ]:
saveTM(tm)

In [ ]:
# to load the TM back from the file do:
with open('tm.nta', 'rb') as f:
    proto2 = TemporalMemoryProto_capnp.TemporalMemoryProto.read(f, traversal_limit_in_words=2**61)
tm = TM.read(proto2)

## Analysis of input

In [ ]:
overlapMatrix = inputAnalysis(allSequences, "random", tm.numberOfColumns())

In [ ]:
# show heatmap of overlap matrix
plt.imshow(overlapMatrix, cmap='spectral', interpolation='nearest')
cb = plt.colorbar()
cb.set_label('Overlap Score')
plt.savefig("overlapScore_heatmap")
plt.close()
# plt.show()

# generate histogram
bins = 60
(n, bins, patches) = plt.hist(overlapMatrix.ravel(), bins, alpha=0.5)

plt.xlabel("Overlap Score")
plt.ylabel("Frequency")
plt.savefig("overlapScore_hist")

plt.xlim(0,0.15)
plt.xlabel("Overlap Score")
plt.ylabel("Frequency")
plt.savefig("overlapScore_hist_ZOOM")
plt.close()

In [ ]:
print colIndicesSmall

In [ ]:
print corrMatrix[0,8]

In [ ]:
np.corrcoef(spikeTrains[1824,:], spikeTrains[1826,:])[0,1]